In [1]:
import numpy as np
from torch import nn,optim
from torch.autograd import Variable
import torch
from torchvision import datasets, transforms # 也是torch的工具，里面封装了更高级的功能
from torch.utils.data import DataLoader

In [2]:
# 训练集
train_dataset = datasets.MNIST(root='./', # 这里选择当前路径下载，意思就是说可以在当前路径MINST文件夹内找到数据，数据是封装好了的
                            train = True, # 下载训练集
                            transform = transforms.ToTensor(), #把数据变成基本的数据类型tensor
                            download = True) # 直接从封装好接口上载入

# 测试集
test_dataset = datasets.MNIST(root='./', # 这里选择当前路径下载
                            train = False, # 下载训练集
                            transform = transforms.ToTensor(), #把数据变成基本的数据类型tensor
                            download = True) # 直接从封装好接口上载入

train_dataset # 这个东西已经被封装成了60000个点，使用Data_loader拿出来用就好
len(train_dataset)


60000

In [3]:
# 批次大小，每次训练选择多少的数据
batch_size = 64 

# 装载训练集
# dataloader相当于创建了一个数据生成器
# 在这个里面数据就已经封装好了（train_loader和test_loader里面）
train_loader = DataLoader(dataset = train_dataset,  # 数据的来源
                          batch_size = batch_size,  # 批次的大小
                          shuffle = True) # 数据是否进行打乱

test_loader = DataLoader(dataset = test_dataset, # 来源于测试集
                         batch_size = batch_size,
                         shuffle = True)

train_loader

In [4]:
# 数据生成器使用方法——在循环中进行使用
for i,data in enumerate(train_loader): #enumerate是为了每一批次的数据增加一个索引i
    inputs, labels = data # data里面包含了要输入的量和输出的标签值
    print(inputs.shape)
    print(labels.shape)
    break
print(labels)
# 64个样本 1个通道（黑白） 28*28的像素
# 标签同样是64个，标签值代表了是哪个数据
len(train_loader) # 938*64 = 60032 刚好枚举完

torch.Size([64, 1, 28, 28])
torch.Size([64])
tensor([4, 8, 5, 8, 4, 1, 8, 7, 3, 3, 6, 5, 1, 1, 4, 6, 0, 8, 4, 8, 9, 8, 0, 7,
        4, 5, 3, 4, 5, 1, 0, 0, 0, 3, 7, 2, 6, 9, 8, 6, 9, 2, 4, 0, 5, 0, 4, 0,
        7, 7, 5, 5, 9, 0, 2, 7, 0, 0, 8, 7, 1, 7, 0, 2])


938

# 对以下网络的一些解释
定义在网络结构里面的每一个背后都内含参数的
全连接层会对不同的样本进行分别处理
但是softmax是可以跨样本进行处理的（可能为了后续一些算法？）
如果dim=0，则会对每一列进行归一化，不符合我们要求，所以要dim=1，完成行归一化

# Dropout的相应用法
Dropout一般在隐藏层中使用，编写代码的时候写在网络结构层编写
在写网络结构时，可以使用封装代码
非封装：self.fc1 = nn.Linear(784,500)
封装版：self.layer1 = nn.Sequential(nn.Linear(784,500), nn.Dropout(p=0.5), nn.Tanh())
然后调用的时候直接用layer1即可

In [5]:
# 定义网络结构 Pytorch，一个网络结构（初始化） 一个网络计算（得到结果）
# 如果要用 Dropout，就要使用多层的网络结构

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__() # 是方法函数就需要用()
        self.layer1 = nn.Sequential(nn.Linear(784,500), nn.Dropout(p=0), nn.Tanh())
        self.layer2 = nn.Sequential(nn.Linear(500,300), nn.Dropout(p=0), nn.Tanh()) # 隐藏层是超参数
        self.layer3 = nn.Sequential(nn.Linear(300,10), nn.Softmax(dim=1)) # 输出层不用Dropout，这些nn的方法第一个字母大写

    def forward(self, x):
        # x 输入的时候维度是[64,1,28,28], 全连接层只能处理2维数据，第一个维度是样本索引，第二个维度是样本内的
        # ([64, 1, 28, 28])->(64, 784)
        x = x.reshape(x.size()[0],-1) #-1代表自适应，变成64行，784列
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x

### 正则化
正则化直接在optimizer里面进行调整即可！
weight_decay 调整L2正则化参数

In [ ]:
# 定义模型（对象化）
model = Net() #对象化也是一个方法
# 定义代价函数
# 这是一个类，你需要实例化这个类！
mse_loss = nn.CrossEntropyLoss() #交叉熵损失
# 定义优化器
# 要传入模型参数和学习率
# 在优化器里面可以加入正则化！！
# weight_decay：设置L2正则化。
LR = 0.5
optimizer = optim.SGD(model.parameters(), LR, weight_decay=0.001)

In [7]:
# 进行模型的训练和测试方法——一边训练一边测试模型效果
def train():
    # 在dropout里面需要额外加上model.train()
    model.train() # 代表是训练状态，此时模型的dropout是起作用的
    for i,data in enumerate(train_loader): # 对迭代器进行分析
        # 循环一次，就会获得一次数据的批次和标签，这很重要！
        inputs, labels = data
        # 获得模型预测结果, 这个结果是[64, 10]的一个矩阵，64是批次，10是概率值
        out = model(inputs)
        
        # 用交叉熵损失，用交叉损失时，函数会内部自动转换成独热编码
        # 其中，out是（64，10）数组，labels是64的一位数组
        # 根据这个函数的用法，两者不需要对齐，只要保证第一个数都是minibatch值即可
        loss = mse_loss(out, labels)
        
        # 梯度清零
        optimizer.zero_grad()
        # 计算梯度
        loss.backward()
        # 修改权值
        optimizer.step()

def test():
    model.eval()
    # 测试集准确率
    correct = 0
    for i,data in enumerate(test_loader): # 每一次都是在对一个训练好的模型进行评估，把所有的测试集跑了一边
        inputs, labels = data
        out = model(inputs)
        # torch.max计算完后得到两个值，获得最大值和最大值所在的位置，dim=1代表对每一行进行操作
        # 这里会对64行每一行都进行操作的
        _, predicted = torch.max(out, 1)
        # 正确个数
        correct += (predicted == labels).sum() # 这里面的true和false会一个一个对比
    print('Test acc:{0}'.format(correct.item()/len(test_dataset)))

    # 训练集准确率
    correct = 0
    for i,data in enumerate(train_loader): # 每一次都是在对一个训练好的模型进行评估，把所有的测试集跑了一边
        inputs, labels = data
        out = model(inputs)
        # torch.max计算完后得到两个值，获得最大值和最大值所在的位置，dim=1代表对每一行进行操作
        # 这里会对64行每一行都进行操作的
        _, predicted = torch.max(out, 1)
        # 正确个数
        correct += (predicted == labels).sum() # 这里面的true和false会一个一个对比
    print('Train acc:{0}'.format(correct.item()/len(train_dataset)))

# Dropout 使用场景
并不一定让准确度变高，在模型比较复杂的情况下，应当使用，防止过拟合

In [8]:
# 循环运行
# 获得了更快的收敛速度
for epoch in range(20):
    print('epoch:', epoch)
    train()
    test()

epoch: 0


KeyboardInterrupt: 

# 这里是一个常见的框架
数据集的载入
定义批次大小和分割数据集（接下去在循环中对这个数据进行训练）
定义网络结构 和 网络运算 def __init__(self) 和 def forward(self, x)
对象化模型 代价函数 优化器
写网络如何训练测试——用两个模块封装好
训练：输出结果 求损失 梯度归零 更新参数
测试：输出结果

一个大循环